The original data creators, depositors or copyright holders, the funders of the
Data Collections (if different) and the UK Data Service/UK Data Archive, and to acknowledge
Crown Copyright where appropriate. The original data creators, depositors or copyright holders, the funders of the Data Collections (if
different) and the UK Data Service/UK Data Archive bear no responsibility for their further
analysis or interpretation.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('data/sdsh2022_sarafu_trainset.csv')
data_users = pd.read_csv('data/sdsh2022_sarafu_users.csv')

In [3]:
print(data_train.shape)
print(data_users.shape)

(5151, 2)
(40842, 8)


for now, keep only data for known labels

In [4]:
data_trxs = pd.read_csv('data/sdsh2022_sarafu_transactions.csv')
data_trxs['is_bonus'] = np.where(data_trxs['source'] == -1, 1, 0)
data_trxs['is_penalty'] = np.where(data_trxs['target'] == -1, 1, 0)
data_trxs['date'] = pd.to_datetime(data_trxs['time']).dt.date

In [5]:
data = data_train.merge(data_users, on='id', how='left')

data = data.merge(
    data_trxs.groupby(['source']).size().reset_index(name='n_transactions_sent'),
    how='left',
    left_on=['id'],
    right_on=['source']
).drop(columns=['source'])

data = data.merge(
    data_trxs.groupby(['target']).size().reset_index(name='n_transactions_received'),
    how='left',
    left_on=['id'],
    right_on=['target']
).drop(columns=['target'])

data = data.merge(
    data_trxs.groupby('source').sum().reset_index().loc[:, ['source', 'is_penalty']],
    how='left',
    left_on=['id'],
    right_on=['source']
).drop(columns=['source'])

data = data.merge(
    data_trxs.groupby('target').sum().reset_index().loc[:, ['target', 'is_bonus']],
    how='left',
    left_on=['id'],
    right_on=['target']
).drop(columns=['target'])

data = data.merge(
    data_trxs.groupby('target')['source'].nunique().reset_index(
    ).rename(columns={"target": "id", "source": "n_partners_received"}),
    how='left',
    on='id'
)

data = data.merge(
    data_trxs.groupby('source')['target'].nunique().reset_index(
    ).rename(columns={"source": "id", "target": "n_partners_sent"}),
    how='left',
    on='id'
)

data = data.merge(
    data_trxs.groupby('source')['date'].first().reset_index(
    ).rename(columns={"source": "id", "date": "first_trx_date"}),
    how='left',
    on='id'
)

data = data.merge(
    data_trxs.groupby('source')['date'].last().reset_index(
    ).rename(columns={"source": "id", "date": "last_trx_date"}),
    how='left',
    on='id'
)

data['n_transactions'] = data['n_transactions_received']+data['n_transactions_sent']
data['n_partners'] = data['n_partners_received']+data['n_partners_sent']
data['registration_date'] = pd.to_datetime(data['registration_time']).dt.date
data['days_before_first_trx'] = data['first_trx_date'] - data['registration_date']

data.rename(columns={"is_bonus": "n_bonuses", "is_penalty": "n_penalties"}, inplace=True)

data['r_time'] = pd.to_datetime(data['registration_time'])
data['days_from_registr'] = pd.to_datetime(data['r_time'].max()) - data['r_time']
data['days_from_registr'] = data['days_from_registr'] / np.timedelta64(1, 'D')
data.drop(columns=['r_time'], inplace=True)

data['count_intensity'] = data['n_transactions']/data['days_from_registr']
data['count_sent_intensity'] = data['n_transactions_sent']/data['days_from_registr']
data['count_received_intensity'] = data['n_transactions_received']/data['days_from_registr']

data['n_transactions_sent'] = data['n_transactions_sent'].fillna(0)
data['n_transactions_received'] = data['n_transactions_received'].fillna(0)
data['n_penalties'] = data['n_penalties'].fillna(0)
data['n_bonuses'] = data['n_bonuses'].fillna(0)
data['n_partners_received'] = data['n_partners_received'].fillna(0)
data['n_partners_sent'] = data['n_partners_sent'].fillna(0)
data['n_transactions'] = data['n_transactions'].fillna(0)
data['n_partners'] = data['n_partners'].fillna(0)
data['days_before_first_trx'] = data['days_before_first_trx'].astype('int64').fillna(0)
data['count_intensity'] = data['count_intensity'].fillna(0)
data['count_sent_intensity'] = data['count_sent_intensity'].fillna(0)
data['count_received_intensity'] = data['count_received_intensity'].fillna(0)
data.drop(columns=['registration_time', 'account_type', 'area_name', 'business_type'], inplace=True)

In [6]:
data.to_csv('data/data_p2', index=False)

what was before penalties and bonuses?
avg_time_between_trxs

is_stage_2: days_past_last_trxs (dpt) more than <n> days 
stage_2_partners_share: sum(partners == is_stage_2)/n_partners